# ME 617 HW2: Airport Terminal Design

#### Authors: Cameron Irmas, Hannah Mankle

_Note: This source code was implemented in Julia. This notebook requires a Julia 1.7.x kernel as well as an environment that includes the necessary dependencies. See the [repo](https://github.com/camirmas/DesignAutomation) for full implementations, testing, and dependency information. All relevant code has been copied into this notebook, so no importing of individual modules is necessary._

### 1

#### Description: With text and code, show how you check if you have achieved the goal.

In order to determine if we have reached the goal, we need to ensure that:
1. The desired number of terminals has been reached
2. No rules are violated anywhere in the matrix

Assuming that we apply the rules as stated at the transition point (children are valid), we can maintain a number of terminals for any given child node, and check against that, avoiding the need for extra computation (though needs to be accounted for in the transition process).

In [2]:
using CSV, DataFrames
using DataStructures

In [3]:
csv_path = "../data/"

"../data/"

In [4]:
function _read_csv(filename)
    println("Reading CSV...")
    m = CSV.read(filename, DataFrame; header=false)
    return Matrix{Int}(m)
end

_read_csv (generic function with 1 method)

In [5]:
struct AirportNode
    matrix
    depth::Int
    terminals::Int
    g::Int
    h::Int
    f::Int

    function AirportNode(matrix, depth, terminals; g=0, h=0)
        return new(matrix, depth, terminals, g, h, g+h)
    end
end

In [6]:
function _get_neighbors(matrix, (row, col); diagonal=true)
    if diagonal
        neighbors = []
        for nrow=row-1:row+1
            for ncol=col-1:col+1
                if nrow == row && ncol == col
                    continue
                else
                    push!(neighbors, matrix[nrow, ncol])
                end
            end
        end
    else
        top = matrix[row-1, col]
        right = matrix[row, col+1]
        bottom = matrix[row+1, col]
        left = matrix[row, col-1]
        neighbors = [top, right, bottom, left]
    end

    return neighbors
end

_get_neighbors (generic function with 1 method)

#### Rule #1
Rule 1 states that at any given index, a 0 can become a 2 if:
 - Bordering a 1 on side
 - Other 3 sides are 0

In [7]:
function check_rule_1(node, (row, col))
    state = node.matrix

    if state[row, col] != 0
        return false
    end

    neighbors = _get_neighbors(node.matrix, (row, col); diagonal=false)

    return sort(neighbors) == [0, 0, 0, 1]
end

check_rule_1 (generic function with 1 method)

####  Rule #2
Rule 2 states that at any given index, a 0 can become a 1 if:
 - Neighbors existing “1”, even if diagonally
 - Doesn’t cause a violation in Rule 1 (handled in transition function)

In [8]:
function check_rule_2(node, (row, col))
    if node.matrix[row, col] != 0
        return false
    end

    neighbors = _get_neighbors(node.matrix, (row, col))

    return 1 in neighbors
end

check_rule_2 (generic function with 1 method)

### 2

#### Description: With text and code, show how you represent the candidates, and transitions for this problem.

Candidates are represented above as `AirportNode`s. We maintain the airport layout (matrix), the current depth, and the number of terminals present at this current state. As mentioned, we can carefully maintain a current number of terminals by incrementing whenever a new terminal is added with a new child node.

Transitions in a typical DFS are represented as a list of functions representing transformations, resulting in child nodes to be added to the stack. In this case, we have two transition functions, one representing 0->1, and the other representing 0->2. Each function will evaluate the current state of the matrix, and any given point.

#### Transition functions
Transitions for adding new walkways

In [9]:
function transition_walkway(node, (row, col))
    state = node.matrix

    if state[row, col] != 0
        return
    end

    if !check_rule_2(node, (row, col))
        return
    end

    new_m = copy(node.matrix)
    new_m[row, col] = 1
    new_node = AirportNode(
        new_m, node.depth+1, node.terminals; g=node.g+1, h=node.h)

    # check the new_m for rule 1 violations
    # new node cannot be directly next to a 2
    neighbors = _get_neighbors(new_node.matrix, (row, col); diagonal=false)

    if 2 in neighbors
        return
    end

    return new_node
end

transition_walkway (generic function with 1 method)

Transition function for adding new terminals 

In [10]:
function transition_terminal(node, (row, col))
    if !check_rule_1(node, (row, col))
        return
    end

    new_m = copy(node.matrix)
    new_m[row, col] = 2
    new_node = AirportNode(
        new_m, node.depth+1, node.terminals+1; g=node.g, h=node.h-1)

    return new_node
end

transition_terminal (generic function with 1 method)

### 3 

#### Description: Write code for and run DFS for the 6 problems above. Set a max depth to 10. This max may or may not be reached in each problem. Show results. What is the average branching factor?

In [11]:
function airport_dfs(terminals, seed_file, max_depth)
    s_fns = [
        transition_terminal,
        transition_walkway
    ]
    nodes = Stack{AirportNode}()
    initial_state = _read_csv(seed_file)
    println("Running Airport Terminal Solver...")
    seed = AirportNode(initial_state, 1, 0)
    push!(nodes, seed)
    last_node = seed
    i = 0

    while !isempty(nodes)
        node = pop!(nodes)
        last_node = node
        state = node.matrix

        i += 1
        if i % 100000 == 0
            println("\n\nIterations: $(i). Stack size: $(length(nodes))")
        end

        if node.depth > max_depth
            continue
        end

        _is_goal(node) = node.terminals == terminals

        if _is_goal(node)
            res = """
                Solution found! Iterations: $(i). Depth: $(node.depth) Stack size: $(length(nodes))
            """
            println(res)
            display(node.matrix)
            return node,i
        end

        # Run walkway and terminal transition functions on current matrix
        # and add valid nodes to the stack.
        rows, cols = size(state)
        for row=1:rows
            for col=1:cols
                for fn in s_fns
                    new_node = fn(node, (row, col))    

                    if isnothing(new_node)
                        continue
                    end

                    push!(nodes, new_node)
                end
            end
        end
    end
    println("failed. final state:\n$(last_node.matrix)")
end

airport_dfs (generic function with 1 method)

In [12]:
node,i = airport_dfs(3, "$(csv_path)A.csv", 10)


Reading CSV...
Running Airport Terminal Solver...


Iterations: 100000. Stack size: 36


Iterations: 200000. Stack size: 37


Iterations: 300000. Stack size: 39


Iterations: 400000. Stack size: 42


Iterations: 500000. Stack size: 27


Iterations: 600000. Stack size: 29


Iterations: 700000. Stack size: 42


Iterations: 800000. Stack size: 31


Iterations: 900000. Stack size: 38


Iterations: 1000000. Stack size: 37


Iterations: 1100000. Stack size: 23


Iterations: 1200000. Stack size: 47


Iterations: 1300000. Stack size: 34


Iterations: 1400000. Stack size: 37


Iterations: 1500000. Stack size: 35


Iterations: 1600000. Stack size: 38


Iterations: 1700000. Stack size: 28


Iterations: 1800000. Stack size: 23


Iterations: 1900000. Stack size: 40


Iterations: 2000000. Stack size: 37


Iterations: 2100000. Stack size: 37


Iterations: 2200000. Stack size: 41


Iterations: 2300000. Stack size: 37


Iterations: 2400000. Stack size: 33


Iterations: 2500000. Stack size: 39


Iterati



Iterations: 26600000. Stack size: 40


Iterations: 26700000. Stack size: 32


Iterations: 26800000. Stack size: 46


Iterations: 26900000. Stack size: 29


Iterations: 27000000. Stack size: 34


Iterations: 27100000. Stack size: 34


Iterations: 27200000. Stack size: 40


Iterations: 27300000. Stack size: 29


Iterations: 27400000. Stack size: 25


Iterations: 27500000. Stack size: 31


Iterations: 27600000. Stack size: 33


Iterations: 27700000. Stack size: 44


Iterations: 27800000. Stack size: 26


Iterations: 27900000. Stack size: 32


Iterations: 28000000. Stack size: 28


Iterations: 28100000. Stack size: 24


Iterations: 28200000. Stack size: 19


Iterations: 28300000. Stack size: 36


Iterations: 28400000. Stack size: 25


Iterations: 28500000. Stack size: 28


Iterations: 28600000. Stack size: 28


Iterations: 28700000. Stack size: 33


Iterations: 28800000. Stack size: 31


Iterations: 28900000. Stack size: 31


Iterations: 29000000. Stack size: 30


Iterations: 29100000. S



Iterations: 52900000. Stack size: 39


Iterations: 53000000. Stack size: 40


Iterations: 53100000. Stack size: 27


Iterations: 53200000. Stack size: 30


Iterations: 53300000. Stack size: 42


Iterations: 53400000. Stack size: 33


Iterations: 53500000. Stack size: 44


Iterations: 53600000. Stack size: 40


Iterations: 53700000. Stack size: 43


Iterations: 53800000. Stack size: 49


Iterations: 53900000. Stack size: 33


Iterations: 54000000. Stack size: 43


Iterations: 54100000. Stack size: 44


Iterations: 54200000. Stack size: 37


Iterations: 54300000. Stack size: 40


Iterations: 54400000. Stack size: 31


Iterations: 54500000. Stack size: 35


Iterations: 54600000. Stack size: 45


Iterations: 54700000. Stack size: 43


Iterations: 54800000. Stack size: 32


Iterations: 54900000. Stack size: 32


Iterations: 55000000. Stack size: 41


Iterations: 55100000. Stack size: 38


Iterations: 55200000. Stack size: 42


Iterations: 55300000. Stack size: 33


Iterations: 55400000. S



Iterations: 79200000. Stack size: 32


Iterations: 79300000. Stack size: 38


Iterations: 79400000. Stack size: 37


Iterations: 79500000. Stack size: 43


Iterations: 79600000. Stack size: 39


Iterations: 79700000. Stack size: 33


Iterations: 79800000. Stack size: 35


Iterations: 79900000. Stack size: 26


Iterations: 80000000. Stack size: 32


Iterations: 80100000. Stack size: 45


Iterations: 80200000. Stack size: 42


Iterations: 80300000. Stack size: 34


Iterations: 80400000. Stack size: 35


Iterations: 80500000. Stack size: 56


Iterations: 80600000. Stack size: 50


Iterations: 80700000. Stack size: 44


Iterations: 80800000. Stack size: 57


Iterations: 80900000. Stack size: 51


Iterations: 81000000. Stack size: 36


Iterations: 81100000. Stack size: 36


Iterations: 81200000. Stack size: 39


Iterations: 81300000. Stack size: 41


Iterations: 81400000. Stack size: 45


Iterations: 81500000. Stack size: 31


Iterations: 81600000. Stack size: 39


Iterations: 81700000. S



Iterations: 105400000. Stack size: 33


Iterations: 105500000. Stack size: 33


Iterations: 105600000. Stack size: 22


Iterations: 105700000. Stack size: 32


Iterations: 105800000. Stack size: 29


Iterations: 105900000. Stack size: 44


Iterations: 106000000. Stack size: 42


Iterations: 106100000. Stack size: 41


Iterations: 106200000. Stack size: 42


Iterations: 106300000. Stack size: 39


Iterations: 106400000. Stack size: 37


Iterations: 106500000. Stack size: 48


Iterations: 106600000. Stack size: 23


Iterations: 106700000. Stack size: 30


Iterations: 106800000. Stack size: 31


Iterations: 106900000. Stack size: 30


Iterations: 107000000. Stack size: 27


Iterations: 107100000. Stack size: 39


Iterations: 107200000. Stack size: 30


Iterations: 107300000. Stack size: 36


Iterations: 107400000. Stack size: 26


Iterations: 107500000. Stack size: 38


Iterations: 107600000. Stack size: 38


Iterations: 107700000. Stack size: 35


Iterations: 107800000. Stack size: 43


9×5 Matrix{Int64}:
 -1  -1  -1  -1  -1
 -1   0   0   0  -1
 -1   1   2   0  -1
 -1   1   0   0  -1
 -1   1   2   0  -1
 -1   1   0   0  -1
 -1   1   2   0  -1
 -1   1   0   0  -1
 -1  -1  -1  -1  -1



Iterations: 131000000. Stack size: 52


Iterations: 131100000. Stack size: 38


Iterations: 131200000. Stack size: 61


Iterations: 131300000. Stack size: 50


Iterations: 131400000. Stack size: 51


Iterations: 131500000. Stack size: 35


Iterations: 131600000. Stack size: 32


Iterations: 131700000. Stack size: 39


Iterations: 131800000. Stack size: 36


Iterations: 131900000. Stack size: 46


Iterations: 132000000. Stack size: 41


Iterations: 132100000. Stack size: 50


Iterations: 132200000. Stack size: 36


Iterations: 132300000. Stack size: 43


Iterations: 132400000. Stack size: 30


Iterations: 132500000. Stack size: 39


Iterations: 132600000. Stack size: 36


Iterations: 132700000. Stack size: 36


Iterations: 132800000. Stack size: 42


Iterations: 132900000. Stack size: 33


Iterations: 133000000. Stack size: 44


Iterations: 133100000. Stack size: 29


Iterations: 133200000. Stack size: 41


Iterations: 133300000. Stack size: 41


Iterations: 133400000. Stack size: 40


(AirportNode([-1 -1 … -1 -1; -1 0 … 0 -1; … ; -1 1 … 0 -1; -1 -1 … -1 -1], 9, 3, 5, -3, 2), 154140633)

In [13]:
display(node.matrix)

9×5 Matrix{Int64}:
 -1  -1  -1  -1  -1
 -1   0   0   0  -1
 -1   1   2   0  -1
 -1   1   0   0  -1
 -1   1   2   0  -1
 -1   1   0   0  -1
 -1   1   2   0  -1
 -1   1   0   0  -1
 -1  -1  -1  -1  -1

Average Branching factor

In [14]:
Beff = exp(log(i)/node.depth) # Expected branching factor 2.27

8.123974862125078

#### Depth First Search Results

With depth first search, the only solution that ran under 20 minutes was the first airport terminal (A.csv). The function ran very slowly but did have a low stack number throughout the process. 

### 4 
#### Description: Write a transition function, g(n), such that the number of walkways are minimized and a heuristic, h(n), to achieve the target number of G gates. Does the heuristic meet the four criteria (zero at goal, monotonically decreasing, admissible, same units). Discuss. (it's okay if it doesn't).

`AirportNode` maintains g(n) and h(n) values. g(n) increases every time a walkway is added to the airport layout, while h(n) starts at target goal and decreases each time a terminal is added. 

h(n) does reach zero when the goal is met. The heuristic is _weakly_ monotonically decreasing due to h(n) remaining at the same value for some iterations. It is difficult to determine if the heuristic is admissible without keeping track of the path of the tree. With that information, however, we can analyze the results similar to Figure 9-19 from the text. One strategy is to have each child keep track of its parents, and to trace the path from the goal node back to the seed, collecting all associated values and observing them graphically. In such a case, even if we find that the heuristic is not admissible (objective function is not strictly monotonic), we would likely still find that our approach reaches optimal solutions. If we did, however, see a concerning trend in the objective function, we could apply the "path-max" approach to force the objective function to be, at worst, weakly monotonic. In terms of units, we did not put any weights on g(n) and h(n), hence both are integer values, though we suspect we would have found improved solutions with a weighting approach favoring the heuristic, particularly for the more complicated cases, E and F. This would not be without its tradeoffs, as emphasizing the heuristic leads to a more greedy algorithm.

### 5
#### Description: Run A* on the 6 problems above. Compare results to 3.

In [15]:
function airport_a_star(terminals, seed_file, max_depth)
    s_fns = [
        transition_terminal,
        transition_walkway
    ]
    nodes = PriorityQueue{AirportNode, Int}()
    initial_state = _read_csv(seed_file)
    println("Running Airport Terminal Solver...")
    seed = AirportNode(initial_state, 1, 0; h=terminals, g=0)
    enqueue!(nodes, seed, seed.f)
    last_node = seed
    i = 0

    while !isempty(nodes)
        node = dequeue!(nodes)
        last_node = node
        state = node.matrix

        i += 1
        if i % 100000 == 0
            println("\n\nIterations: $(i). Stack size: $(length(nodes)). g: $(node.g), h: $(node.h)")
        end

        if node.depth > max_depth
            continue
        end

        _is_goal(node) = node.terminals == terminals

        if _is_goal(node)
            res = """
                Solution found! Iterations: $(i). g: $(node.g). h: $(node.h). \
                Depth: $(node.depth) Queue size: $(length(nodes))
            """
            println(res)
            display(node.matrix)
            return node
        end

        # Run walkway and terminal transition functions on current matrix
        # and add valid nodes to the stack.
        rows, cols = size(state)
        for row=1:rows
            for col=1:cols
                for fn in s_fns
                    new_node = fn(node, (row, col))    

                    if isnothing(new_node)
                        continue
                    end

                    enqueue!(nodes, new_node, new_node.f)
                end
            end
        end
    end
    println("failed. final state:\n$(last_node.matrix)")
end


airport_a_star (generic function with 1 method)

In [16]:
node = airport_a_star(3, "$(csv_path)A.csv", 10)

9×5 Matrix{Int64}:
 -1  -1  -1  -1  -1
 -1   0   0   0  -1
 -1   1   2   0  -1
 -1   1   0   0  -1
 -1   1   2   0  -1
 -1   1   0   0  -1
 -1   1   2   0  -1
 -1   0   0   0  -1
 -1  -1  -1  -1  -1

Reading CSV...
Running Airport Terminal Solver...
    Solution found! Iterations: 195. g: 4. h: 0. Depth: 8 Queue size: 566



AirportNode([-1 -1 … -1 -1; -1 0 … 0 -1; … ; -1 0 … 0 -1; -1 -1 … -1 -1], 8, 3, 4, 0, 4)

In [17]:
node = airport_a_star(4, "$(csv_path)B.csv", 10)

7×7 Matrix{Int64}:
 -1  -1  -1  -1  -1  -1  -1
 -1   0   0   1   0   0  -1
 -1   0   2   1   2   0  -1
 -1   0   0   1   0   0  -1
 -1   0   2   1   2   0  -1
 -1   0   0   0   0   0  -1
 -1  -1  -1  -1  -1  -1  -1

Reading CSV...
Running Airport Terminal Solver...
    Solution found! Iterations: 74. g: 3. h: 0. Depth: 8 Queue size: 237



AirportNode([-1 -1 … -1 -1; -1 0 … 0 -1; … ; -1 0 … 0 -1; -1 -1 … -1 -1], 8, 4, 3, 0, 3)

In [18]:
node = airport_a_star(6, "$(csv_path)C.csv", 10)

7×9 Matrix{Int64}:
 -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1   0   0   0   0   0   0   0  -1
 -1   0   0   2   0   2   0   0  -1
 -1   0   2   1   1   1   2   0  -1
 -1   0   0   2   0   2   0   0  -1
 -1   0   0   0   0   0   0   0  -1
 -1  -1  -1  -1  -1  -1  -1  -1  -1

Reading CSV...
Running Airport Terminal Solver...
    Solution found! Iterations: 24852. g: 2. h: 0. Depth: 9 Queue size: 70464



AirportNode([-1 -1 … -1 -1; -1 0 … 0 -1; … ; -1 0 … 0 -1; -1 -1 … -1 -1], 9, 6, 2, 0, 2)

In [19]:
node = airport_a_star(9, "$(csv_path)D.csv", 18)

10×10 Matrix{Int64}:
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1   0   0   0   0   0   0   0   0  -1
 -1   1   2   0   0   2   0   0   0  -1
 -1   1   0   2   0   1   2   0   0  -1
 -1   0   1   1   1   1   0   0   0  -1
 -1   0   2   0   2   0   1   2   0  -1
 -1   0   0   0   0   2   1   0   0  -1
 -1   0   0   0   0   0   2   0   0  -1
 -1   0   0   0   0   0   0   0   0  -1
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1

Reading CSV...
Running Airport Terminal Solver...
    Solution found! Iterations: 2846. g: 8. h: 0. Depth: 18 Queue size: 7191



AirportNode([-1 -1 … -1 -1; -1 0 … 0 -1; … ; -1 0 … 0 -1; -1 -1 … -1 -1], 18, 9, 8, 0, 8)

In [20]:
node = airport_a_star(9, "$(csv_path)E.csv", 19)

10×10 Matrix{Int64}:
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1   1   0   0  -1  -1  -1  -1  -1  -1
 -1   1   2   0   0  -1  -1  -1  -1  -1
 -1   1   0   2   0   0  -1  -1  -1  -1
 -1   0   1   1   1   2   0  -1  -1  -1
 -1   0   2   0   1   0   2   0  -1  -1
 -1   0   0   2   1   1   1   2   0  -1
 -1   0   0   0   2   0   2   0   0  -1
 -1   0   0   0   0   0   0   0   0  -1
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1

Reading CSV...
Running Airport Terminal Solver...
    Solution found! Iterations: 69209. g: 9. h: 0. Depth: 19 Queue size: 279257



AirportNode([-1 -1 … -1 -1; -1 1 … -1 -1; … ; -1 0 … 0 -1; -1 -1 … -1 -1], 19, 9, 9, 0, 9)

In [21]:
node = airport_a_star(11, "$(csv_path)F.csv", 23)

10×13 Matrix{Int64}:
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1   0   1   0   0   0  -1  -1  -1   0   0   0  -1
 -1   0   0   1   2   0  -1  -1   0   0   0   0  -1
 -1   0   2   1   0   0  -1  -1   0   2   0   0  -1
 -1   0   0   1   2   0  -1  -1   0   1   2   0  -1
 -1   0   2   1   0   2   0  -1   1   0   0   0  -1
 -1   0   0   0   1   1   1   1   1   2   0   0  -1
 -1   0   0   0   2   0   2   0   2   0   0   0  -1
 -1   0   0   0   0   0   0   0   0   0   0   0  -1
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1

Reading CSV...
Running Airport Terminal Solver...
    Solution found! Iterations: 21251. g: 11. h: 0. Depth: 23 Queue size: 59277



AirportNode([-1 -1 … -1 -1; -1 0 … 0 -1; … ; -1 0 … 0 -1; -1 -1 … -1 -1], 23, 11, 11, 0, 11)

#### A* Results

Overall, A* performed much faster than depth first search and was able to find solutions for all of the problems under 20 minutes. For some of the more complex problems, we increased the max depth of the tree to find a solution. Interestingly, while the algorithm finds solutions without the use of a max depth parameter (indicating completeness), when no max depth was supplied, we occasionally saw a worse solution result (i.e. case C, where the seed has a walkway in the very middle of the grid). 

As expected, A* also had a larger queue than DFS. This memory usage, while relatively small for the cases provided, highlights the exponential nature of space and time complexity for A* algorithms, and provides context for the usage of algorithms like Beam Search and IDA*.

### 6
#### Description: Based on your results. What are the limits of the size of problem that can be solved. What other approaches could be tried to solve complex problems like this?

Space and time limitations are intrinsic to the performance of A* algorithms. We've seen through this exercise that, even for relatively small problems, the exponential nature of worst-case space and time complexity plays a large role in performance. Given that problems like this one do not scale well, we can consider other algorithms that reduce space and time constraints at the cost of completeness and optimality guarantees. For example, if we know our space limit and the problem has a small branching factor, it could be benefical to use a Beam Search method. Beam search uses linear time and space, and works well for deep trees. This will result in a more practical solution at the possible expense of finding the optimal solution. 